In [27]:
import json
from pymatgen import Lattice, Structure, Molecule, Composition
from pymatgen.io.vasp import Poscar
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer, PointGroupAnalyzer
import numpy as np
from monty.json import MontyEncoder, MontyDecoder
from monty.serialization import loadfn, dumpfn
import pandas as pd
from itertools import chain
import os
import os.path
import pprint
from pymatgen.analysis.elasticity.elastic import ElasticTensor
from pymatgen.io.pwscf import PWInput, PWOutput
from pymatgen.io.cif import CifParser
import glob
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

#returns kpoint grid size based on an 8x8 kpoint density for MoS2
def get_kpt_gridsize(struct):
    
    kpt_den = 25

    a = struct.lattice.a
    b = struct.lattice.b
    c = struct.lattice.c

    latlist =[a,b,c]
    kptlist = [0,0,0]


    for i, kpoint in enumerate(kptlist):
        for val in range(1,20):
                if latlist[i]*val < kpt_den:
                    kptlist[i] = val
                else:
                    kptlist[i] = val
                    break
    return kptlist

def generate_pw_input(struct, name, calctype = 'vc-relax', path = './QE_relax_inputs/'):
    
    filname = name
    kpts = get_kpt_gridsize(struct)
    
    species_list = []
    for i in struct.species:
        if str(i) not in species_list:
            species_list.append(str(i))
    species_list
    pseudo = {}
    for el in species_list:
        pseudo[el] = el + '.LDA-PW-paw.UPF'

    control ={'calculation': calctype,
        'restart_mode':'from_scratch',
        'prefix':'bn',
        'pseudo_dir' : '/gpfs/u/home/TSDM/TSDMluck/barn/Pseudo/UPF/PAW',
        'outdir': '/gpfs/u/home/TSDM/TSDMkrbc/scratch/Zeolites/' + filname,
        'wf_collect':True,
        'forc_conv_thr':.000389,
        'verbosity':'high',
        'max_seconds': 42900,
        'nstep':200
    }
    system = { 'ecutwfc' : 30,
        'occupations':'smearing',
        'smearing' : 'gaussian',
        'degauss':0.005
    }
    electrons = {    'mixing_mode' : 'plain',
        'mixing_beta' : 0.4,
        'conv_thr' :  1.0E-8,
        'scf_must_converge':False
    }
    ions = {    'ion_dynamics' : 'bfgs'
    }
    if calctype == 'vc-relax':
        cell = {'cell_dofree' : 'all'}
    kpoints_grid = kpts
    PWInput(struct,pseudo=pseudo,control=control,system=system,electrons=electrons,
            ions=ions, cell=cell, kpoints_grid =kpoints_grid).write_file(path +filname + ".relax.in")
    print("Wrote " + filname + ".relax.in")

def poscar_wr(struct,name):
    fileout = Poscar(struct)
    filename = 'VASP_files_conventional/' + name + '_' + struct.formula.replace(" ", "") + '.vasp'
    fileout.write_file(filename,direct=False)
    print("Wrote " + filename)

In [170]:
path = 'Relaxed_Vasp\\'
pathlist = []
for filename in glob.iglob(path + '*'):
     pathlist.append('%s' % filename)

structdict = {}
for file in pathlist:
    try:
        structdict[file.replace(path,'').replace('.vasp','')] = Poscar.from_file(file).structure
        print(file, " worked")
    except:
        print(file, " failed")
        pass
structdict

Relaxed_Vasp\Ag1Bi1P2Se6.vasp  worked
Relaxed_Vasp\Al1N1.vasp  worked
Relaxed_Vasp\As4S6.vasp  worked
Relaxed_Vasp\As4Se6.vasp  worked
Relaxed_Vasp\B1N1.vasp  worked
Relaxed_Vasp\B2C3N6.vasp  worked
Relaxed_Vasp\Bi1O2.vasp  worked
Relaxed_Vasp\Bi1Te1Br1.vasp  worked
Relaxed_Vasp\Bi1Te1Cl1.vasp  worked
Relaxed_Vasp\Bi1Te1I1.vasp  worked
Relaxed_Vasp\Cd1In1Ga1S4.vasp  worked
Relaxed_Vasp\Cu1Bi1P2Se6.vasp  worked
Relaxed_Vasp\Cu1Br1.vasp  worked
Relaxed_Vasp\Ga1Ag1P2Se6.vasp  worked
Relaxed_Vasp\Ga1N1.vasp  worked
Relaxed_Vasp\Ga2H2O4.vasp  worked
Relaxed_Vasp\Ga2S2.vasp  worked
Relaxed_Vasp\Ga2Se2.vasp  worked
Relaxed_Vasp\Ge2S2.vasp  worked
Relaxed_Vasp\Ge3Sb2O9.vasp  worked
Relaxed_Vasp\In1Cu1P2Se6.vasp  worked
Relaxed_Vasp\In2Se2.vasp  worked
Relaxed_Vasp\K1C1N1.vasp  worked
Relaxed_Vasp\Mo1S2.vasp  worked
Relaxed_Vasp\Mo1Se2.vasp  worked
Relaxed_Vasp\Nb3Br8.vasp  worked
Relaxed_Vasp\Nb3I8.vasp  worked
Relaxed_Vasp\Nb6Ge2Te12.vasp  worked
Relaxed_Vasp\Rb3Mo2Cl9.vasp  worked
Relaxed_Va

{'Ag1Bi1P2Se6': Structure Summary
 Lattice
     abc : 6.7479981670001061 6.7479981089854615 23.597957000000001
  angles : 90.0 90.0 120.00000504472558
  volume : 930.58259004700597
       A : 6.7479981670000004 -1.1939999999999999e-06 0.0
       B : -3.3739985350000001 5.8439380869999997 0.0
       C : 0.0 0.0 23.597957000000001
 PeriodicSite: Ag (-0.0000, 3.8960, 11.3830) [0.3333, 0.6667, 0.4824]
 PeriodicSite: Bi (0.0000, 0.0000, 11.9894) [-0.0000, -0.0000, 0.5081]
 PeriodicSite: P (3.3740, 1.9480, 10.6998) [0.6667, 0.3333, 0.4534]
 PeriodicSite: P (3.3740, 1.9480, 12.9880) [0.6667, 0.3333, 0.5504]
 PeriodicSite: Se (1.2677, 1.8198, 10.0023) [0.3436, 0.3114, 0.4239]
 PeriodicSite: Se (4.5382, 0.1880, 10.0023) [0.6886, 0.0322, 0.4239]
 PeriodicSite: Se (4.3161, 3.8362, 10.0023) [0.9678, 0.6564, 0.4239]
 PeriodicSite: Se (-1.2613, 2.1917, 13.5903) [0.0006, 0.3750, 0.5759]
 PeriodicSite: Se (2.1066, 3.6557, 13.5903) [0.6250, 0.6256, 0.5759]
 PeriodicSite: Se (2.5288, -0.0035, 13.5903) [

In [58]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen import MPRester
for key in structdict:
    #print(key, SpacegroupAnalyzer(structdict[key]).get_crystal_system())
    #print(key, SpacegroupAnalyzer(structdict[key]).get_point_group_symbol())
    x,y = structdict[key].lattice.angles[0], structdict[key].lattice.angles[1]
    if x != 90 and y != 90:
        print(key, x, y)
    #print(key, SpacegroupAnalyzer(structdict[key]).get_space_group_number())

As4S6 89.9007012269 90.4800248609
Sb2As2O6 90.0025371888 90.7177913408
Sb2N2 91.3043428524 91.5511500365
Ta3S1Br7 89.6781064821 90.0000003164
Te2H4O8 93.8478002432 90.0000003223
V1Ag1P2Se6 86.7813064004 83.3802945984
Zn1W1O4 89.9979673855 83.0629265791


In [56]:
SpacegroupAnalyzer(structdict['As4S6']).get_primitive_standard_structure()

Structure Summary
Lattice
    abc : 5.0208100997105367 11.243830660326596 20.476499645713584
 angles : 89.900701226864498 89.519975139094953 83.762834318678799
 volume : 1149.0794584749731
      A : 5.0208100997105367 0.0 0.0
      B : 1.2215770139133131 11.177275048829166 0.0
      C : 0.17155040023459653 0.016949985602053696 20.475773999999998
PeriodicSite: As (4.7669, 10.9582, 10.0288) [0.6943, 0.9797, 0.4898]
PeriodicSite: As (3.5355, 3.1044, 9.6951) [0.6206, 0.2770, 0.4735]
PeriodicSite: As (3.5668, 8.0705, 10.5499) [0.5173, 0.7213, 0.5152]
PeriodicSite: As (1.0334, 5.5354, 10.6813) [0.0677, 0.4944, 0.5217]
PeriodicSite: S (2.3879, 1.5415, 10.9305) [0.4240, 0.1371, 0.5338]
PeriodicSite: S (3.0661, 9.7035, 9.0496) [0.3845, 0.8675, 0.4420]
PeriodicSite: S (4.7394, 9.5830, 11.8576) [0.7158, 0.8565, 0.5791]
PeriodicSite: S (4.2485, 4.3061, 11.4868) [0.7335, 0.3844, 0.5610]
PeriodicSite: S (1.7950, 4.3764, 8.8827) [0.2476, 0.3909, 0.4338]
PeriodicSite: S (5.1808, 7.0732, 9.2167) [0.862

In [30]:
keylist = []
for key in structdict:
    keylist.append(key)

In [169]:
import ast
from functools import reduce
df = pd.read_csv('ipa.csv', delimiter=";")
df = df[df['material'].isin(keylist)]
series_list = []

for i, val in enumerate(list(df['data'])):
    #df['data'][i] = ast.literal_eval(val)
    s1 = pd.Series(ast.literal_eval(val))
    s1['MATERIAL'] = list(df['material'])[i]
    series_list.append(s1.to_frame().T)

df = reduce(lambda x, y: pd.merge(x, y, how = 'outer'), series_list)
df = df[df['IS_METALLIC'] == False]
df = df.reindex_axis(['MATERIAL','BAND_GAP', 'FERMI_ENERGY', 'GAP_TYPE', 'HOMO_BAND_NUM', 'HOMO_ENERGY',
       'HOMO_KPT_NUM', 'IS_METALLIC', 'LUMO_ENERGY', 'LUMO_KPT_NUM'], axis=1)
df

,MATERIAL,BAND_GAP,FERMI_ENERGY,GAP_TYPE,HOMO_BAND_NUM,HOMO_ENERGY,HOMO_KPT_NUM,IS_METALLIC,LUMO_ENERGY,LUMO_KPT_NUM
0,Ag1Bi1P2Se6,1.449735,-1.808219,INDIRECT,40.0,-2.390286,5.0,False,-0.940551,1.0
1,Al1N1,2.959902,-2.632145,INDIRECT,8.0,-3.710517,21.0,False,-0.750615,1.0
2,As4S6,2.289074,-2.454304,INDIRECT,28.0,-4.333726,17.0,False,-2.044653,1.0
3,As4Se6,1.773107,-2.311243,INDIRECT,28.0,-3.746376,8.0,False,-1.973269,1.0
4,B1N1,4.785515,-1.653103,INDIRECT,4.0,-4.138593,30.0,False,0.646922,1.0
5,B2C3N6,2.592857,-4.013141,DIRECT,24.0,-5.604520,1.0,False,-3.011663,1.0
7,Bi1Te1Br1,1.603044,-2.226936,INDIRECT,19.0,-2.799097,12.0,False,-1.196053,1.0
8,Bi1Te1Cl1,0.465926,-3.382069,DIRECT,19.0,-3.603804,1.0,False,-3.137878,1.0
9,Bi1Te1I1,1.519473,-1.578529,INDIRECT,24.0,-2.578485,22.0,False,-1.059011,1.0
10,Cd1In1Ga1S4,0.493569,0.054313,INDIRECT,35.0,-0.202417,2.0,False,0.291152,1.0


In [263]:
import json
from pymatgen import Lattice, Structure, Molecule, Composition
from pymatgen.io.vasp import Poscar
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer, PointGroupAnalyzer
import numpy as np
from monty.json import MontyEncoder, MontyDecoder
from monty.serialization import loadfn, dumpfn
import pandas as pd
from itertools import chain
import os
import os.path
import pprint
from pymatgen.analysis.elasticity.elastic import ElasticTensor
from pymatgen.io.pwscf import PWInput, PWOutput
from pymatgen.io.cif import CifParser
import glob
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer


def get_kpt_gridsize(struct, kpt_den):
    
    #kpt_den = 38
    #kpt_den = 66.5
    
    a = struct.lattice.a
    b = struct.lattice.b
    c = struct.lattice.c

    latlist =[a,b,c]
    kptlist = [0,0,0]


    for i, kpoint in enumerate(kptlist):
        for val in range(1,40):
                if latlist[i]*val < kpt_den:
                    kptlist[i] = val
                else:
                    kptlist[i] = val
                    break
    kptlist[2] = 1
    return kptlist

#Generate Quantum Espresso Input File
def generate_pw_input(struct, name, nbnd=0, kpt_den = 38, calctype = 'scf',path = './SCF_INP/'):
    
    filname = name
    kpts = get_kpt_gridsize(struct, kpt_den)
    
    species_list = []
    for i in struct.species:
        if str(i) not in species_list:
            species_list.append(str(i))
    species_list
    pseudo = {}
    for el in species_list:
        pseudo[el] = el + '_ONCV_PBE_sr.upf'

    control ={'calculation': calctype,
        'restart_mode':'from_scratch',
        'prefix':'bn',
        'pseudo_dir' : '/home/beachk2/PSEUDO/upf_files/PBE/ONCVPSP-master/sg15/',
        'outdir': '/scratch/beachk2/Jarvis/IPA/' + filname,
        'wf_collect':True,
        'forc_conv_thr':1.0E-4,
        'verbosity':'high'
    }
    system = { 'ecutwfc' : 70,
        'occupations':'smearing',
        'smearing' : 'gaussian',
        'degauss':0.005,
        'force_symmorphic' : True,
        'nbnd' : nbnd
    }
    electrons = {    'mixing_mode' : 'plain',
        'mixing_beta' : 0.7,
        'conv_thr' :  1.0E-8
    }
    ions = {    'ion_dynamics' : 'bfgs'
    }
    cell = {'cell_dofree' : '2Dxy'}
    kpoints_grid = kpts
    
    PWInput(struct,pseudo=pseudo,control=control,system=system,electrons=electrons, 
            ions=ions, cell=cell, kpoints_grid =kpoints_grid).write_file(path +filname + "." + calctype + ".in")
    print("Wrote " + filname + "." + calctype + ".in")

In [245]:
for num in df.index:
    generate_pw_input(struct=structdict[df['MATERIAL'][num]], 
                      name=df['MATERIAL'][num],
                      nbnd=int(df['HOMO_BAND_NUM'][num])+10,
                      calctype='scf',
                      path=)

Wrote Ag1Bi1P2Se6.scf.in
Wrote Al1N1.scf.in
Wrote As4S6.scf.in
Wrote As4Se6.scf.in
Wrote B1N1.scf.in
Wrote B2C3N6.scf.in
Wrote Bi1Te1Br1.scf.in
Wrote Bi1Te1Cl1.scf.in
Wrote Bi1Te1I1.scf.in
Wrote Cd1In1Ga1S4.scf.in
Wrote Cu1Bi1P2Se6.scf.in
Wrote Cu1Br1.scf.in
Wrote Ga1Ag1P2Se6.scf.in
Wrote Ga1N1.scf.in
Wrote Ga2H2O4.scf.in
Wrote Ga2S2.scf.in
Wrote Ga2Se2.scf.in
Wrote Ge2S2.scf.in
Wrote Ge3Sb2O9.scf.in
Wrote In1Cu1P2Se6.scf.in
Wrote In2Se2.scf.in
Wrote K1C1N1.scf.in
Wrote Mo1S2.scf.in
Wrote Mo1Se2.scf.in
Wrote Sb2As2O6.scf.in
Wrote Sb2N2.scf.in
Wrote Sb2Se2I2.scf.in
Wrote Sc1Ag1P2S6.scf.in
Wrote Sc1Ag1P2Se6.scf.in
Wrote Si4As8.scf.in
Wrote Sn4S4.scf.in
Wrote Ta3S1Br7.scf.in
Wrote Te1O3.scf.in
Wrote Te2H4O8.scf.in
Wrote Te2Mo1.scf.in
Wrote V1Ag1P2Se6.scf.in
Wrote W1S2.scf.in
Wrote W1Se2.scf.in
Wrote Zn1Cl2.scf.in
Wrote Zn1W1O4.scf.in


In [274]:
import json
from pymatgen import Lattice, Structure, Molecule, Composition
from pymatgen.io.vasp import Poscar
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer, PointGroupAnalyzer
import numpy as np
from monty.json import MontyEncoder, MontyDecoder
from monty.serialization import loadfn, dumpfn
import pandas as pd
from itertools import chain
import os
import os.path
import pprint
from pymatgen.analysis.elasticity.elastic import ElasticTensor
from pymatgen.io.pwscf import PWInput, PWOutput
from pymatgen.io.cif import CifParser
import glob
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import ast
from functools import reduce

def get_kpt_gridsize(struct, kpt_den):
    
    #kpt_den = 38
    #kpt_den = 66.5
    
    a = struct.lattice.a
    b = struct.lattice.b
    c = struct.lattice.c

    latlist =[a,b,c]
    kptlist = [0,0,0]


    for i, kpoint in enumerate(kptlist):
        for val in range(1,40):
                if latlist[i]*val < kpt_den:
                    kptlist[i] = val
                else:
                    kptlist[i] = val
                    break
    kptlist[2] = 1
    return kptlist

#Generate Quantum Espresso Input File
def generate_pw_input(struct, name, nbnd=0, kpt_den = 38, calctype = 'scf',path = './SCF_INP/'):
    
    filname = name
    kpts = get_kpt_gridsize(struct, kpt_den)
    
    species_list = []
    for i in struct.species:
        if str(i) not in species_list:
            species_list.append(str(i))
    species_list
    pseudo = {}
    for el in species_list:
        pseudo[el] = el + '_ONCV_PBE_sr.upf'

    control ={'calculation': calctype,
        'restart_mode':'from_scratch',
        'prefix':'bn',
        'pseudo_dir' : '/home/beachk2/PSEUDO/upf_files/PBE/ONCVPSP-master/sg15/',
        'outdir': '/scratch/beachk2/Jarvis/IPA/' + filname,
        'wf_collect':True,
        'forc_conv_thr':1.0E-4,
        'verbosity':'high'
    }
    system = { 'ecutwfc' : 70,
        'occupations':'smearing',
        'smearing' : 'gaussian',
        'degauss':0.005,
        'force_symmorphic' : True,
        'nbnd' : nbnd
    }
    electrons = {    'mixing_mode' : 'plain',
        'mixing_beta' : 0.7,
        'conv_thr' :  1.0E-8
    }
    ions = {    'ion_dynamics' : 'bfgs'
    }
    cell = {'cell_dofree' : '2Dxy'}
    kpoints_grid = kpts
    
    PWInput(struct,pseudo=pseudo,control=control,system=system,electrons=electrons, 
            ions=ions, cell=cell, kpoints_grid =kpoints_grid).write_file(path + calctype + ".in")
    #print("Wrote " + filname + "." + calctype + ".in")


path = 'Relaxed_Vasp\\'
pathlist = []
for filename in glob.iglob(path + '*'):
     pathlist.append('%s' % filename)

structdict = {}
for file in pathlist:
    try:
        structdict[file.replace(path,'').replace('.vasp','')] = Poscar.from_file(file).structure
        #print(file, " worked")
    except:
        #print(file, " failed")
        pass

df = pd.read_csv('ipa.csv', delimiter=";")
df = df[df['material'].isin(keylist)]
series_list = []

for i, val in enumerate(list(df['data'])):
    #df['data'][i] = ast.literal_eval(val)
    s1 = pd.Series(ast.literal_eval(val))
    s1['MATERIAL'] = list(df['material'])[i]
    series_list.append(s1.to_frame().T)

df = reduce(lambda x, y: pd.merge(x, y, how = 'outer'), series_list)
df = df[df['IS_METALLIC'] == False]
df = df.reindex_axis(['MATERIAL','BAND_GAP', 'FERMI_ENERGY', 'GAP_TYPE', 'HOMO_BAND_NUM', 'HOMO_ENERGY',
       'HOMO_KPT_NUM', 'IS_METALLIC', 'LUMO_ENERGY', 'LUMO_KPT_NUM'], axis=1)

In [285]:
xylist = [[1.0,1.0],[0.0,1.0],[1.0,0.0]]
for num in df.index:
    for xy in xylist:
        
        x = xy[0]
        y = xy[1]

        band_range = 6
        
        if int(df['HOMO_BAND_NUM'][num]) < band_range+1:
            minbnd = 1
        else:
            minbnd = int(df['HOMO_BAND_NUM'][num])-band_range
        maxbnd = int(df['HOMO_BAND_NUM'][num])+band_range
        
        name = df['MATERIAL'][num]


        if x == 1.0 and y == 0.0:
            fieldstr = "100"    
        elif x == 0.0 and y == 1.0:
            fieldstr = "010"
        elif x == 1.0 and y == 1.0:
            fieldstr = "110"        
        path = "Lumen_INP/" + name + "/" + fieldstr + "/INP/"
        path2 = "Lumen_INP/" + name + "/" + fieldstr + "/"
        os.makedirs(path, exist_ok=True)


        generate_pw_input(struct=structdict[df['MATERIAL'][num]], 
                          name=df['MATERIAL'][num],
                          nbnd=int(df['HOMO_BAND_NUM'][num])+10,
                          kpt_den=38,
                          calctype='scf',
                          path=path)
        generate_pw_input(struct=structdict[df['MATERIAL'][num]], 
                          name=df['MATERIAL'][num],
                          nbnd=int(df['HOMO_BAND_NUM'][num])+10,
                          kpt_den=66.5,
                          calctype='nscf',
                          path=path)
        
        with open("Lumen_INP/lumen_template", "rt") as fin:
            with open(path + "lumen.in", "wt") as fout:
                for line in fin:
                    fout.write(line.replace(
                        'LOW_BAND', str(minbnd)).replace(
                        'HI_BAND', str(maxbnd)).replace(
                        'X_FIELD', str(x)).replace(
                        'Y_FIELD', str(y)))
        
        with open("Lumen_INP/lumen.run", "rt") as fin:
            with open(path + "lumen.run", "wt") as fout:
                for line in fin:
                    fout.write(line.replace(
                        'INSERT_FIELD', fieldstr).replace(
                        'INSERT_MAT', df['MATERIAL'][num]))
        
        with open("Lumen_INP/scf.run", "rt") as fin:
            with open(path + "scf.run", "wt") as fout:
                for line in fin:
                    fout.write(line)
                    
        with open("Lumen_INP/Params", "rt") as fin:
            with open(path + "Params", "wt") as fout:
                for line in fin:
                    fout.write(line)
        
        with open("Lumen_INP/runlumen", "rt") as fin:
            with open(path2 + "runlumen", "wt") as fout:
                for line in fin:
                    fout.write(line)
        
        with open("Lumen_INP/ypp_template", "rt") as fin:
            with open(path + "ypp.in", "wt") as fout:
                for line in fin:
                    fout.write(line.replace(
                        'X_FIELD', str(x)).replace(
                        'Y_FIELD', str(y)))
        fin.close()
        fout.close()